Лабораторна робота 2 
Предмет: Дискретна математика
Семестр: 2
Виконавці: Бабенко Аліна, Тхір Назар.

Розподіл роботи між учасниками команди:
Алгоритм Гафмана - Бабенко Аліна
LZ77 - Тхір Назар
LZW - Бабенко Аліна

Нижче приведені алгоритми за порядком, їх перевірки та коефіцієнти стиску: LZ77, LZW , алгоритм Гафмана, Deflate.

In [45]:
import copy
class lz77:
    def __init__(self, buffer_length=255) -> None:
        """
        Init a lz77 class with a buffer length.
        Default set to 255.
        """
        self.buffer_length = buffer_length

    def read(self, file_path):
        """
        Read data from a file.
        """
        with open(file_path, encoding="UTF-8") as file:
            self.data = file.read()
            self.orig = copy.deepcopy(self.data)
    
    def write(self, data, file_path):
        """
        Write data into a file.
        """
        with open(file_path,'w', encoding="UTF-8") as file:
            file.write(data)

    def find_substring(self, buffer, data):
        """
        Find the longest common substring in the buffer.
        """
        length = 0
        offset = 0
        while True:
            it = buffer.find(data[:length + 1])
            if it != -1:
                offset = len(buffer[it:])
                length += 1
                if data[:length] == data[:length + 1]:
                    return (length, (offset, length, None))
                continue
            break
        next = data[length]
        return (length, (offset, length, next))

    def compress(self):
        """
        Compress the data.
        """
        ans = []
        buffer = ""
        while self.data:
            length, code = self.find_substring(buffer, self.data)
            buffer += self.data[:length + 1]
            if len(buffer) > self.buffer_length:
                buffer = buffer[len(buffer) - self.buffer_length:]
            self.data = self.data[length + 1:]
            ans.append(code)
        self.compressed = ans

    def decompress(self):
        """
        Decompress the data.
        """
        buffer = ""
        ans = ""
        for (offset, length, next) in self.compressed:
            if next == None:
                ans +=  buffer[len(buffer) - offset:len(buffer) - offset + length]
                break
            added = buffer[len(buffer) - offset:len(buffer) - offset + length] + next
            ans += added
            buffer += added
            if len(buffer) > self.buffer_length:
                buffer = buffer[len(buffer) - self.buffer_length:]
        self.decompressed = ans

        

    def encode(self, file_path):
        """
        Encode data from a file and write into the new file.
        """
        self.read(file_path)
        self.compress()
        self.write(str(self.compressed),f"{file_path.split('.')[0]}_compressed.txt")

    def decode(self, file_path):
        """
        Decode a code from a file.
        """
        self.decompress()
        self.write(self.decompressed, f"{file_path.split('.')[0]}_decompressed.txt")

    def calculate_compression(self):
        """
        Calculate compression.
        """
        new = len(self.compressed) * 3
        orig = len(self.orig)
        return new/orig





In [71]:
lz = lz77()
lz.encode('drive.txt')
lz.decode('drive_compressed.txt')
with open('drive.txt', encoding="UTF-8") as file:
            orig = file.read()
with open('drive_compressed_decompressed.txt', encoding="UTF-8") as file:
            new = file.read()
assert orig == new
print(f'Compression rate: {round(lz.calculate_compression() * 100, 3)}%')

Compression rate: 67.784%


Висновок: цей алгоритм варто використовувати коли неоюхідно закодувати дані, що повторюються(циклічні).

In [72]:
class LZW: 
    def read(self, file_path):
        """
        Read data from a file.
        """
        with open(file_path, encoding="UTF-8") as file:
            self.data = file.read()
        
    def write(self, data, file_path):
        """
        Write data into a file.
        """
        with open(file_path,'w', encoding="UTF-8") as file:
            file.write(data)
    
    def compress(self):
        """
        Encode the data.
        """
        dictionary = {chr(i): i for i in range(256)}
        prev = ""
        ans = []
        for elem in self.data:
            new = prev + elem
            if new in dictionary:
                prev = new
            else:
                ans.append(dictionary[prev])
                dictionary[new] = len(dictionary)
                prev = elem
        if prev:
            ans.append(dictionary[prev])
        self.compressed = ans


    def decompress(self):
        """
        Decode the data.
        """
        dictionary = {i: chr(i) for i in range(256)}
        prev = chr(self.compressed[0])
        ans = prev
        for code in self.compressed[1:]:
            if code in dictionary:
                val = dictionary[code]
            else:
                val = prev + prev[0]
            ans += val
            dictionary[len(dictionary)] = prev + val[0]
            prev = val
        self.decompressed = ans

    def encode(self, file_path):
        """
        Encode data from a file and write into the new file.
        """
        self.read(file_path)
        self.compress()
        self.write(str(self.compressed),f"{file_path.split('.')[0]}_compressed.txt")

    def decode(self, file_path):
        """
        Decode a code from a file.
        """
        self.decompress()
        self.write(self.decompressed, f"{file_path.split('.')[0]}_decompressed.txt")
    
    def calculate_compression(self):
        """
        Calculate compression.
        """
        new = len(self.compressed) * 2
        orig = len(self.data)
        return new/orig


In [73]:
lzw = LZW()
lzw.encode('drive.txt')
lzw.decode('drive_compressed.txt')
with open('drive.txt', encoding="UTF-8") as file:
            orig = file.read()
with open('drive_compressed_decompressed.txt', encoding="UTF-8") as file:
            new = file.read()
assert orig == new
print(f'Compression rate: {round(lzw.calculate_compression() * 100, 3)}%')

Compression rate: 30.577%


Висновок:
    Краще використовувати LZW, коли потрібно стиснути довгу послідовність символів. LZW ефективно працює з даними, які мають більш рівномірний розподіл частот символів, що дозволяє зберігати більше інформації в меншому обсязі.

In [74]:
class Node():
    """
    Huffman tree node.
    """
    def __init__(self, character = None, freq = 0, left = None, right = None) -> None:
        self.character = character
        self.freq = freq
        self.left = left
        self.right = right

class HuffmanTree():
    def __init__(self) -> None:
        self.encoder = {}
        self.decoder = {}
    def read(self, file_path):
        """
        Read data from a file.
        """
        with open(file_path, encoding="UTF-8") as file:
            self.data = file.read()
        self.build_tree()

    def write(self, data, file_path):
        """
        Write data into a file.
        """
        with open(file_path,'w', encoding="UTF-8") as file:
            file.write(data)
    
    def build_tree(self):
        """
        Build the Huffman tree from the given data.
        """
        frequency = {}
        for character in self.data:
            frequency[character] = frequency.get(character, 0) + 1
        sorted_freq = {k: v for k, v in sorted(frequency.items(), key=lambda item: item[1])}
        vertexes = [Node(character=character, freq=freq) for character, freq in sorted_freq.items()]
        while len(vertexes) > 1:
            vertexes.sort(key=lambda x: x.freq)
            right = vertexes.pop(0)
            left = vertexes.pop(0)
            parent_vert = Node(freq=left.freq + right.freq, left=left, right=right)
            vertexes.append(parent_vert)
        self.root = vertexes[0]
        self.get_codes(self.root)

    def get_codes(self, node, code=''):
        """
        Make codes for each character.
        """
        if node is None:
            return
        if node.character is not None:
            node.code = code
            self.encoder[node.character] = code
            self.decoder[code] = node.character
        self.get_codes(node.left, code + '0')
        self.get_codes(node.right, code + '1')

    def encode(self, file_path):
        """
        Encode the given data using the Huffman tree.
        """
        self.read(file_path)
        ans = ''
        for character in self.data:
            ans += self.encoder[character]
        self.coded = ans
        self.write(str(self.coded),f"{file_path.split('.')[0]}_compressed.txt")

    def decode(self, file_path):
        """
        Decode the given data using the Huffman tree.
        """
        ans = ''
        cur = ''
        for elem in self.coded:
            cur += elem
            if cur in self.decoder:
                ans += self.decoder[cur]
                cur = ''
        self.decoded = ans
        self.write(str(self.decoded),f"{file_path.split('.')[0]}_decompressed.txt")
    
    def calculate_compression(self):
        """
        Calculate compression.
        """
        new = int(len(self.coded) / 8)
        orig = len(self.data)
        return new/orig

In [75]:
huf = HuffmanTree()
huf.encode('drive.txt')
huf.decode('drive_compressed.txt')
with open('drive.txt', encoding="UTF-8") as file:
            orig = file.read()
with open('drive_compressed_decompressed.txt', encoding="UTF-8") as file:
            new = file.read()
assert orig == new
print(f'Compression rate: {round(huf.calculate_compression() * 100, 3)}%')



Compression rate: 43.923%


Висновок: 
    Краще використовувати кодування Гафмана, коли необхідно стиснути дані, щоб зменшити їх розмір. Кодування Гафмана дає кращі результати при використанні для стиснення даних, що мають велику кількість повторюваних символів.

In [76]:
class Deflate:
    def __init__(self):
        self.lz = lz77()
        self.huf = HuffmanTree()
    def read(self, file_path):
        """
        Read data from a file.
        """
        with open(file_path, encoding="UTF-8") as file:
            self.data = file.read()
        
    def write(self, data, file_path):
        """
        Write data into a file.
        """
        with open(file_path,'w', encoding="UTF-8") as file:
            file.write(data)
    
    def encode(self, file_path):
        """
        Encode the data.
        """
        self.lz.encode(file_path)     
        self.huf.encode(f'{file_path.split(".")[0]}_compressed.txt') 
        self.write(self.huf.coded, f'{file_path.split(".")[0]}_compressed_deflated.txt')

    def decode(self, file_path_in, file_path_out):
        """
        Decode the data.
        """
        data = self.read(file_path_in)
        huf.decoded = data
        huf.decode(file_path_in)
        self.lz.decompressed = self.read(f'{file_path_in.split(".")[0]}_decompressed.txt')
        self.lz.decode(file_path_out)
    
    def calculate_compression(self):
        return self.lz.calculate_compression() * self.huf.calculate_compression()
        
        
        


In [77]:
defl = Deflate()
defl.encode('drive.txt')
defl.decode('drive_compressed_deflated.txt','inflated.txt')
with open('drive.txt', encoding="UTF-8") as file:
            orig = file.read()
with open('inflated_decompressed.txt', encoding="UTF-8") as file:
            new = file.read()
assert orig == new
print(f'Compression rate: {round(defl.calculate_compression() * 100, 3)}%')

Compression rate: 31.374%


Замість того щоб будувати графіки (потрібно багато даних та час) ми вирішили навести статистику:

In [78]:
lz = lz77()
lz.encode('drive.txt')
print(f'Lz77 small compression rate: {round(lz.calculate_compression() * 100, 3)}%')

lz = lz77()
lz.encode('2mb.txt')
print(f'Lz77 big compression rate: {round(lz.calculate_compression() * 100, 3)}%')

lz = lz77()
lz.encode('cyclic.txt')
print(f'Lz77 cyclic compression rate: {round(lz.calculate_compression() * 100, 3)}%')

lzw = LZW()
lzw.encode('drive.txt')
print(f'LZW small compression rate: {round(lzw.calculate_compression() * 100, 3)}%')

lzw = LZW()
lzw.encode('2mb.txt')
print(f'LZW big compression rate: {round(lzw.calculate_compression() * 100, 3)}%')

lzw = LZW()
lzw.encode('cyclic.txt')
print(f'LZW cyclic compression rate: {round(lzw.calculate_compression() * 100, 3)}%')

huf = HuffmanTree()
huf.encode('drive.txt')
print(f'Huffman small compression rate: {round(huf.calculate_compression() * 100, 3)}%')

huf = HuffmanTree()
huf.encode('2mb.txt')
print(f'Huffman big compression rate: {round(huf.calculate_compression() * 100, 3)}%')

huf = HuffmanTree()
huf.encode('cyclic.txt')
print(f'Huffman cyclic compression rate: {round(huf.calculate_compression() * 100, 3)}%')

defl = Deflate()
defl.encode('drive.txt')
print(f'Deflate small compression rate: {round(defl.calculate_compression() * 100, 3)}%')

defl = Deflate()
defl.encode('2mb.txt')
print(f'Deflate big compression rate: {round(defl.calculate_compression() * 100, 3)}%')

defl = Deflate()
defl.encode('cyclic.txt')
print(f'Deflate cyclic compression rate: {round(defl.calculate_compression() * 100, 3)}%')

Lz77 small compression rate: 67.784%
Lz77 big compression rate: 87.753%
Lz77 cyclic compression rate: 2.069%
LZW small compression rate: 30.577%
LZW big compression rate: 16.729%
LZW cyclic compression rate: 11.737%
Huffman small compression rate: 43.923%
Huffman big compression rate: 52.844%
Huffman cyclic compression rate: 53.931%
Deflate small compression rate: 31.374%
Deflate big compression rate: 40.336%
Deflate cyclic compression rate: 0.936%


Чітко видно сильні та слабкі сторони кожного з алгоритмів.